<a href="https://colab.research.google.com/github/CattoYT/ValorantCommsBot/blob/rewrite/YoloV8_Training_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
!pip install ultralytics

#!pip install -U ipywidgets # mainly for kaggle

!pip install pyngrok
from pyngrok import conf, ngrok
from google.colab import userdata

#conf.get_default().auth_token = "" # use with kaggle
conf.get_default().auth_token = userdata.get('NGROKToken')

public_url = ngrok.connect("80", domain="cheaply-caring-pup.ngrok-free.app").public_url# change this domain



print(public_url)


In [3]:
# @title Download Dataset here

from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO
dataseturl = "" # @param {type:"string"}
datasetname = None

with urlopen(dataseturl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        !mkdir dataset
        zfile.extractall("dataset")

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
from ultralytics import YOLO

resume = True # @param {type:"boolean"}
if resume:
    #pass the training directory pls
    resumeDir = "/content/drive/MyDrive/Yolov8/ValorantV8" # @param {type:"string"}
    model = YOLO(resumeDir + "/weights/last.pt")
else:
    model = YOLO("yolov8s.pt")


In [ ]:

# @title Default title text
dataDirectory = "dataset" # @param {type:"string"}
import os
os.environ['WANDB_MODE'] = 'disabled'
%load_ext tensorboard

if resume:
    %tensorboard --port=80 --logdir /content/drive/MyDrive/Yolov8/
    model.train(resume=True, device=0)

else:
    %tensorboard --port=80 --logdir runs/detect/train

    results = model.train(data=f"/content/dataset/data.yaml", #
                      epochs=400,
                      imgsz=640,
                      cache=True,
                      batch=0.969, # set to 0.90 for colab
                      patience=25,
                      save_period=10,
                      verbose=True,
                      device=1,
                      #device=[0,1],
                      project="/content/drive/MyDrive/Yolov8",
                      name="ValorantV8"
                     )

     41/250      14.3G      1.021     0.5404     0.9459         86        640:  59%|█████▊    | 64/109 [01:28<00:56,  1.26s/it]

In [ ]:
# @title Export Model
model.export(format="onnx", dynamic=True) # no idea if engine works properly in this environment.
#If you want to export to tensorrt, make sure this is running on the correct device.
#For now, onnx will do ig

In [ ]:
def exportModel(directory):
    from google.colab import files

    !zip -r {directory}.zip {directory}
    files.download(f"{directory}.zip")

if resume == True:
    exportModel(resumeDir+"/weights/")
else:
    exportModel("runs/detect/train")